In [1]:
SUPPORT_TRASHOLD=0.005
import pyspark.sql.functions as F
from pyspark.sql import Row
import time
import re

def removeTimestamp(row):
    find = re.compile(r'\d{4}-\d{2}-\d{2}')
    start = re.search(find,row).start()
    return row[0:start]

def spliteToUserIdAndUserSearch(row):
    row = row.split("\t", 1)
    row[1] =  row[1].rstrip('\t')
    return (int(row[0]),row[1])

def uniqueList(line):
    uniqueSearches = set(line[1])
    newLine = [line[0],list(uniqueSearches)]
    return newLine

start = time.time()
log_txt=sc.textFile("Test3.txt")
header = log_txt.first()


log_txt = log_txt.filter(lambda line: line != header)
logSearch = log_txt.map(lambda line: spliteToUserIdAndUserSearch(removeTimestamp(line))).distinct()

print(logSearch.take(2))

[(142, 'rentdirect.com'), (142, 'staple.com')]


In [2]:
totalOfTransactions = logSearch.groupByKey().count()

In [3]:
print(totalOfTransactions)

4


In [6]:
# remove the user id -> return only query 
all_queries = logSearch.map(lambda line: line[1])
# count how much time query is show for all the user divide by number of users
rdd_query_count = all_queries.map(lambda q: (q, 1) ).reduceByKey(lambda c1,c2: c1+c2 ) \
                                                    .map(lambda x: (x[0], x[1] / totalOfTransactions)) \
                                                    .filter(lambda x: x[1] > SUPPORT_TRASHOLD)

# rdd_query_count is list of queries that pass the thrasholds of support
rdd_query_count.take(10)


[('rentdirect.com', 0.5),
 ('staple.com', 0.5),
 ('www.newyorklawyersite.com', 0.5),
 ('207 ad2d 530', 0.5),
 ('frankmellace.com', 0.5),
 ('ucs.ljx.com', 0.5),
 ('attornyleslie.com', 0.5),
 ('merit release appearance', 0.5),
 ('www.bonsai.wbff.org', 0.5),
 ('loislaw.com', 0.5)]

In [7]:
# get the set of all queries
validItems = rdd_query_count.map(lambda x:x[0]).collect()
print(validItems)

['rentdirect.com', 'staple.com', 'www.newyorklawyersite.com', '207 ad2d 530', 'frankmellace.com', 'ucs.ljx.com', 'attornyleslie.com', 'merit release appearance', 'www.bonsai.wbff.org', 'loislaw.com', 'ameriprise.com', "p; .; p;' p; ' ;' ;';", 'asiansexygoddess.com', 'ask.com', 'wellsfargo.com', 'www.tabiecummings.com', 'wanttickets.com', 'yahoo.com', 'www.ngo-quen.org', 'myspace.co', 'myspace.com', 'chasebadkids.net', 'lottery', 'susheme', 'united.com', 'mizuno.com', 'buddylis', 'bestasiancompany.com', 'weather.com', '-', 'vietnam', 'googl', 'www.prescriptionfortime.com', 'westchester.gov', 'space.comhttp', 'dfdf', 'vaniqa.comh', 'www.collegeucla.edu', 'www.elaorg', 'broadway.vera.org', 'vera.org', 'rapny.com', 'whitepages.com']


In [8]:
def include_queries_by_support_thrasholds(inValidList,validItems):
    return [s for s in inValidList if s in validItems]
user_query = logSearch.groupByKey().mapValues(list).filter(lambda kv: len(kv[1]) > 1)  
user_query = user_query.map(lambda t: (t[0],include_queries_by_support_thrasholds(t[1],validItems)))
user_query.take(2)


[(142,
  ['rentdirect.com',
   'staple.com',
   'www.newyorklawyersite.com',
   '207 ad2d 530',
   'frankmellace.com',
   'ucs.ljx.com',
   'attornyleslie.com',
   'merit release appearance',
   'www.bonsai.wbff.org',
   'loislaw.com',
   'www.prescriptionfortime.com',
   'westchester.gov',
   'space.comhttp',
   'dfdf',
   'vaniqa.comh',
   'www.collegeucla.edu',
   'www.elaorg',
   'broadway.vera.org',
   'vera.org',
   'rapny.com',
   'whitepages.com']),
 (52,
  ['rentdirect.com',
   'staple.com',
   'www.newyorklawyersite.com',
   '207 ad2d 530',
   'frankmellace.com',
   'ucs.ljx.com',
   'attornyleslie.com',
   'merit release appearance',
   'www.bonsai.wbff.org',
   'loislaw.com',
   'www.prescriptionfortime.com',
   'westchester.gov',
   'space.comhttp',
   'dfdf',
   'vaniqa.comh',
   'www.collegeucla.edu',
   'www.elaorg',
   'broadway.vera.org',
   'vera.org',
   'rapny.com',
   'whitepages.com'])]

In [9]:
def get_all_pairs(arr):
        result = []
        for p1 in range(len(arr)):
                for p2 in range(p1+1,len(arr)):
                        result.append((arr[p1],arr[p2]))
        return result
    
all_queries_pairs_tuples = user_query.map(lambda kv: kv[1]).flatMap(lambda arr: get_all_pairs(arr))
all_queries_pairs_tuples.take(50)
# userId | a, b ,c ,d
# 1| 1,1,0,0->a,b
# 2|0,1,1,0->b,c


[('rentdirect.com', 'staple.com'),
 ('rentdirect.com', 'www.newyorklawyersite.com'),
 ('rentdirect.com', '207 ad2d 530'),
 ('rentdirect.com', 'frankmellace.com'),
 ('rentdirect.com', 'ucs.ljx.com'),
 ('rentdirect.com', 'attornyleslie.com'),
 ('rentdirect.com', 'merit release appearance'),
 ('rentdirect.com', 'www.bonsai.wbff.org'),
 ('rentdirect.com', 'loislaw.com'),
 ('rentdirect.com', 'www.prescriptionfortime.com'),
 ('rentdirect.com', 'westchester.gov'),
 ('rentdirect.com', 'space.comhttp'),
 ('rentdirect.com', 'dfdf'),
 ('rentdirect.com', 'vaniqa.comh'),
 ('rentdirect.com', 'www.collegeucla.edu'),
 ('rentdirect.com', 'www.elaorg'),
 ('rentdirect.com', 'broadway.vera.org'),
 ('rentdirect.com', 'vera.org'),
 ('rentdirect.com', 'rapny.com'),
 ('rentdirect.com', 'whitepages.com'),
 ('staple.com', 'www.newyorklawyersite.com'),
 ('staple.com', '207 ad2d 530'),
 ('staple.com', 'frankmellace.com'),
 ('staple.com', 'ucs.ljx.com'),
 ('staple.com', 'attornyleslie.com'),
 ('staple.com', 'merit

In [10]:
def sort_small_list(arr):
    if(arr[0] <= arr[1]):
        return arr
    return [arr[1],arr[0]]
# the sort is for (a,b) (b,a) = > (a,b) (a,b) => ((a,b),2)
all_queries_tuples_sorted = all_queries_pairs_tuples.map(lambda kv: sort_small_list(list(kv))) \
                                                .map(lambda arr: (arr[0],arr[1]) )

all_queries_pairs_tuples_count = all_queries_tuples_sorted.map(lambda kv: (kv,1)) \
                                                    .reduceByKey(lambda c1,c2: c1+c2 )\
                                                    .filter(lambda kv: kv[1] > 1) \
                                                    .map(lambda x: (x[0], x[1] / totalOfTransactions)) 


all_queries_pairs_tuples_count.take(10)


[(('rentdirect.com', 'staple.com'), 0.5),
 (('rentdirect.com', 'www.newyorklawyersite.com'), 0.5),
 (('207 ad2d 530', 'rentdirect.com'), 0.5),
 (('frankmellace.com', 'rentdirect.com'), 0.5),
 (('rentdirect.com', 'ucs.ljx.com'), 0.5),
 (('attornyleslie.com', 'rentdirect.com'), 0.5),
 (('merit release appearance', 'rentdirect.com'), 0.5),
 (('rentdirect.com', 'www.bonsai.wbff.org'), 0.5),
 (('loislaw.com', 'rentdirect.com'), 0.5),
 (('staple.com', 'www.newyorklawyersite.com'), 0.5)]

In [11]:
rdd_queries_tuples_cartesian = all_queries_pairs_tuples_count.cartesian(rdd_query_count)
rdd_queries_tuples_cartesian.take(3)

[((('rentdirect.com', 'staple.com'), 0.5), ('rentdirect.com', 0.5)),
 ((('rentdirect.com', 'staple.com'), 0.5), ('staple.com', 0.5)),
 ((('rentdirect.com', 'staple.com'), 0.5), ('www.newyorklawyersite.com', 0.5))]

In [12]:
# this command calculate XUY/X by taking all lines that ((x ,y , number of suply(xUy)),(z ,number of suply(z))) when z ==x
rdd_join_left = rdd_queries_tuples_cartesian.filter(lambda lr: lr[0][0][0] == lr[1][0]) \
                                            .map(lambda lr: (lr[0][0][0],lr[0][0][1],float(lr[0][1]) / lr[1][1]))

print(rdd_join_left.take(5))
# this command take XUY/Y
print('')
rdd_join_right = rdd_queries_tuples_cartesian.filter(lambda lr: lr[0][0][1] == lr[1][0])\
                                             .map(lambda lr: (lr[0][0][1], lr[0][0][0], float(lr[0][1]) / lr[1][1]))
print(rdd_join_right.take(5))
print('')


[('rentdirect.com', 'staple.com', 1.0), ('rentdirect.com', 'www.newyorklawyersite.com', 1.0), ('207 ad2d 530', 'rentdirect.com', 1.0), ('rentdirect.com', 'ucs.ljx.com', 1.0), ('frankmellace.com', 'rentdirect.com', 1.0)]

[('staple.com', 'rentdirect.com', 1.0), ('rentdirect.com', '207 ad2d 530', 1.0), ('www.newyorklawyersite.com', 'rentdirect.com', 1.0), ('rentdirect.com', 'frankmellace.com', 1.0), ('rentdirect.com', 'attornyleslie.com', 1.0)]



In [13]:
end = time.time()
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end - start))
print("elapsed time: %s" % elapsed_time)

elapsed time: 00:00:52


In [14]:
#######################
####### TASK 1 ########
#######################

rdd_query_conf = sc.union([rdd_join_left, rdd_join_right])
print(rdd_query_conf.take(7))

[('rentdirect.com', 'staple.com', 1.0), ('rentdirect.com', 'www.newyorklawyersite.com', 1.0), ('207 ad2d 530', 'rentdirect.com', 1.0), ('rentdirect.com', 'ucs.ljx.com', 1.0), ('frankmellace.com', 'rentdirect.com', 1.0), ('attornyleslie.com', 'rentdirect.com', 1.0), ('merit release appearance', 'rentdirect.com', 1.0)]


In [16]:
#######################
####### TASK 2  #######
#######################
rddQueryConfDF = sqlContext.createDataFrame(rdd_query_conf, ["X", "Y","CONFIDENCE"])
rddQueryConfDF.coalesce(1).write.format('com.databricks.spark.csv').save('/home/kfir/Desktop/Ex2/Final/my.csv',header = 'true')

In [17]:
#########################
####### TASK 4.a ########
#########################

def filterConfidence(line,Conf):
    if line[2] >=Conf:
        return(True)
    return(False)

x_y_conf06 = rdd_query_conf.filter(lambda line: filterConfidence(line,0.6))
x_y_conf08 = x_y_conf06.filter(lambda line: filterConfidence(line,0.8))
x_y_conf09 = x_y_conf08.filter(lambda line:filterConfidence(line,0.9))
print('the amount of related queries for 0.6 confidence')
print(x_y_conf06.count())

print('the amount of related queries for 0.8 confidence')
print(x_y_conf08.count())

print('the amount of related queries for 0.9 confidence')
print(x_y_conf09.count())


the amount of related queries for 0.6 confidence
420
the amount of related queries for 0.8 confidence
420
the amount of related queries for 0.9 confidence
420


the amount of related queries for 0.6 confidence
420
the amount of related queries for 0.8 confidence
420
the amount of related queries for 0.9 confidence
420


+--------------------+--------------------+----------+
|                   X|                   Y|CONFIDENCE|
+--------------------+--------------------+----------+
|      rentdirect.com|          staple.com|       1.0|
|      rentdirect.com|www.newyorklawyer...|       1.0|
|        207 ad2d 530|      rentdirect.com|       1.0|
|      rentdirect.com|         ucs.ljx.com|       1.0|
|    frankmellace.com|      rentdirect.com|       1.0|
|   attornyleslie.com|      rentdirect.com|       1.0|
|merit release app...|      rentdirect.com|       1.0|
|      rentdirect.com| www.bonsai.wbff.org|       1.0|
|          staple.com|www.newyorklawyer...|       1.0|
|          staple.com|         ucs.ljx.com|       1.0|
|        207 ad2d 530|          staple.com|       1.0|
|    frankmellace.com|          staple.com|       1.0|
|   attornyleslie.com|          staple.com|       1.0|
|         loislaw.com|      rentdirect.com|       1.0|
|merit release app...|          staple.com|       1.0|
|         